In [1]:
import torch.nn as nn

import torch

import nltk

In [2]:
doc = "They think he's a good president because he's done things they like: appointing conservatives to the court and cutting taxes, for example. But every other normal Republican would have done the exact same things, made actual deals to get much more, and they'd have left out all the ridiculous drama that keeps Trump's approval so low and his accomplishments so meager." 

In [3]:
sentences = nltk.sent_tokenize(doc)

In [4]:
import pandas as pd

df = pd.DataFrame(columns = ["sentence"])

df["sentence"] = sentences
df["sentence"] = df.sentence.map(lambda s :  s.lower())

In [5]:
df['words'] = df.sentence.map(nltk.word_tokenize)

In [6]:
df.words

0    [they, think, he, 's, a, good, president, beca...
1    [but, every, other, normal, republican, would,...
Name: words, dtype: object

In [7]:
max_sentence_len = df.words.map(len).max()

In [8]:
import itertools

vocab = list(set(itertools.chain.from_iterable(df.words.tolist())))

In [9]:
vocab

['left',
 'same',
 'deals',
 'he',
 'more',
 'low',
 'a',
 'for',
 'things',
 'drama',
 'accomplishments',
 'every',
 'have',
 'so',
 'approval',
 'because',
 'and',
 'but',
 'actual',
 'think',
 'president',
 'example',
 'the',
 'good',
 'normal',
 'court',
 'would',
 'out',
 'all',
 'get',
 'they',
 'republican',
 "'d",
 'done',
 'conservatives',
 ',',
 'keeps',
 'meager',
 'exact',
 'like',
 'to',
 "'s",
 '.',
 'that',
 'other',
 'his',
 'taxes',
 ':',
 'cutting',
 'much',
 'ridiculous',
 'trump',
 'appointing',
 'made']

In [10]:
import re
?re.match

In [11]:
re.match("\w+",vocab[1])

<_sre.SRE_Match object; span=(0, 4), match='same'>

In [12]:
def matcher(word):
    return re.match("\w+", word)

vocab = list(filter(matcher, itertools.chain.from_iterable(df.words)))

In [13]:
vocab += ["<unk>"]

In [14]:
vocab

['they',
 'think',
 'he',
 'a',
 'good',
 'president',
 'because',
 'he',
 'done',
 'things',
 'they',
 'like',
 'appointing',
 'conservatives',
 'to',
 'the',
 'court',
 'and',
 'cutting',
 'taxes',
 'for',
 'example',
 'but',
 'every',
 'other',
 'normal',
 'republican',
 'would',
 'have',
 'done',
 'the',
 'exact',
 'same',
 'things',
 'made',
 'actual',
 'deals',
 'to',
 'get',
 'much',
 'more',
 'and',
 'they',
 'have',
 'left',
 'out',
 'all',
 'the',
 'ridiculous',
 'drama',
 'that',
 'keeps',
 'trump',
 'approval',
 'so',
 'low',
 'and',
 'his',
 'accomplishments',
 'so',
 'meager',
 '<unk>']

In [15]:
index2vocab = {
    index: word
    for index, word in enumerate(vocab)
}

vocab2index = {
    word: index
    for index, word in enumerate(vocab)
}

In [16]:
vocab2index

{'they': 42,
 'think': 1,
 'he': 7,
 'a': 3,
 'good': 4,
 'president': 5,
 'because': 6,
 'done': 29,
 'things': 33,
 'like': 11,
 'appointing': 12,
 'conservatives': 13,
 'to': 37,
 'the': 47,
 'court': 16,
 'and': 56,
 'cutting': 18,
 'taxes': 19,
 'for': 20,
 'example': 21,
 'but': 22,
 'every': 23,
 'other': 24,
 'normal': 25,
 'republican': 26,
 'would': 27,
 'have': 43,
 'exact': 31,
 'same': 32,
 'made': 34,
 'actual': 35,
 'deals': 36,
 'get': 38,
 'much': 39,
 'more': 40,
 'left': 44,
 'out': 45,
 'all': 46,
 'ridiculous': 48,
 'drama': 49,
 'that': 50,
 'keeps': 51,
 'trump': 52,
 'approval': 53,
 'so': 59,
 'low': 55,
 'his': 57,
 'accomplishments': 58,
 'meager': 60,
 '<unk>': 61}

In [17]:
def get_word_index(word):
    index = vocab2index.get(
        word,
        vocab2index["<unk>"]
    )
    return index

df["word_indices"] = df.words.map(
    lambda words: list(map(get_word_index, words))
)

In [18]:
df.word_indices

0    [42, 1, 7, 61, 3, 4, 5, 6, 7, 61, 29, 33, 42, ...
1    [22, 23, 24, 25, 26, 27, 43, 29, 47, 31, 32, 3...
Name: word_indices, dtype: object

In [19]:
from torch.autograd import Variable

def list2var(l):
    print(l)
    tensor = torch.LongTensor(l)
    return Variable(tensor)

variables = df.word_indices.map(list2var).tolist()

[42, 1, 7, 61, 3, 4, 5, 6, 7, 61, 29, 33, 42, 11, 61, 12, 13, 37, 47, 16, 56, 18, 19, 61, 20, 21, 61]
[22, 23, 24, 25, 26, 27, 43, 29, 47, 31, 32, 33, 61, 34, 35, 36, 37, 38, 39, 40, 61, 56, 42, 61, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 61, 53, 59, 55, 56, 57, 58, 59, 60, 61]


In [20]:
from torch.nn.utils.rnn import pad_sequence

seq = pad_sequence(variables, batch_first=True)

In [21]:
embedding = nn.Embedding(num_embeddings=len(vocab), embedding_dim=100)

In [22]:
a = embedding(seq[0])
b = embedding(seq[1])

In [23]:
c = torch.stack([a,b], dim=0)
c.shape

torch.Size([2, 44, 100])

In [24]:
seq

tensor([[42,  1,  7, 61,  3,  4,  5,  6,  7, 61, 29, 33, 42, 11, 61, 12, 13, 37,
         47, 16, 56, 18, 19, 61, 20, 21, 61,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [22, 23, 24, 25, 26, 27, 43, 29, 47, 31, 32, 33, 61, 34, 35, 36, 37, 38,
         39, 40, 61, 56, 42, 61, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 61, 53,
         59, 55, 56, 57, 58, 59, 60, 61]])

In [25]:
class WordLSTM(nn.Module):
    
    def __init__(self, embedding_dim, vocab_size, lstm_size=100, bidirectional=False):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, lstm_size, bidirectional=bidirectional, batch_first=True)
    
    def seq_to_embedding(self, seq):
        '''
        Use the padded sequence to get embeddings
        '''
        embeds = []
        
        for s in seq:
            embeds.append(self.embedding(s))
            
        return torch.stack(embeds, dim=0)
    
    def forward(self, input, hidden, cell):
        batch = self.seq_to_embedding(input)
        output, (hidden,cell) = self.lstm(batch, (hidden, cell))
        
        return output, hidden, cell
        

In [26]:
lstm = WordLSTM(100, len(vocab), bidirectional=True)

In [27]:
h_0 = torch.zeros(2, 2, 100)
c_0 = torch.zeros(2, 2, 100)
o, h, cs = lstm(seq, h_0,c_0)
o.shape

torch.Size([2, 44, 200])

- 2 sentences
- 44 words in each
- 100 dim of each word
- PyTorch LSTM is only concerned with the last dimension (100)
- For word attention all sentences( each sentence is a batch ) is padded

## Word Attention

In [34]:
class WordAttn(nn.Module):
    
    def __init__(self, hidden_size):
        super().__init__()
        
        self.linear = nn.Linear(hidden_size, hidden_size)
        self.activation = nn.functional.tanh
        self.word_context = nn.Parameter(torch.randn(hidden_size,1))
    
    def forward(self, word_outputs):
        
        o = self.linear(word_outputs)
        o = self.activation(o)
        o = torch.matmul(o, self.word_context)
        o = torch.mul(o, word_outputs)
        o = torch.sum(o, dim=1) ## Sum along the words
        
        return o
        

In [35]:
attn = WordAttn(200)
s_vec = attn(o)

In [36]:
s_vec.shape

torch.Size([2, 200])

In [37]:
s_vec

tensor([[ 3.3239e+00,  1.7234e+00,  2.4380e+00,  2.3324e+00,  2.8235e-01,
          5.0313e+00,  6.4516e+00, -2.9920e+00,  2.4822e-01,  3.5835e+00,
         -2.9361e+00,  3.6338e+00, -2.2503e+00, -2.6444e-01, -2.7912e+00,
          8.9907e-01,  3.0398e+00,  1.1397e+00,  2.0828e+00, -6.8375e-01,
         -1.6930e+00,  1.6707e+00,  2.7948e+00, -1.2604e+00,  3.1361e+00,
         -4.8111e+00,  1.7177e+00, -4.5576e-01, -2.9207e+00,  1.8649e+00,
         -5.1941e-01, -2.2394e+00,  1.4448e-01,  1.3136e+00,  5.0659e+00,
          1.1054e+00, -3.1563e+00, -3.5277e+00, -1.7769e+00,  2.6171e+00,
         -2.2232e+00,  1.0031e-01, -1.1388e+00, -1.3125e+00, -6.0528e-01,
         -1.0570e+00, -2.9848e+00, -3.5599e-01, -2.7809e+00, -1.2778e+00,
          1.4552e+00,  3.9595e-01, -1.5901e+00,  6.2451e-01,  1.9781e+00,
          2.5839e+00,  2.2140e-01,  3.2322e-01,  1.2642e+00, -2.2887e+00,
         -6.2425e-01,  3.7624e-01,  4.1016e+00,  1.4536e+00,  1.2577e+00,
          1.3359e+00,  5.4895e+00, -3.

## Sentence LSTM

In [43]:
class SentenceLSTM(nn.Module):
    def __init__(self, sentence_vec_size=200 ,hidden_size=100, bidirectional=True):
        super().__init__()
        self.lstm = nn.LSTM(input_size=sentence_vec_size, hidden_size=hidden_size, bidirectional=bidirectional, batch_first=True)
    
    def forward(self, input, hidden, cell):
        o = self.lstm(input, (hidden, cell))
        return o

In [44]:
sentence_lstm = SentenceLSTM()

In [48]:
s_vec.unsqueeze(dim=0).shape

torch.Size([1, 2, 200])

In [53]:
h_0_s = torch.zeros(2,1,100)
c_0_s = torch.zeros(2,1,100)
sentence_output, (h_s, c_s) = sentence_lstm(s_vec.unsqueeze(dim=0), h_0_s, c_0_s)

In [54]:
sentence_output.shape

torch.Size([1, 2, 200])

## Sentence Attention

In [64]:
class SentAttn(nn.Module):
    
    def __init__(self, hidden_size):
        super().__init__()
        
        self.linear = nn.Linear(hidden_size, hidden_size)
        self.activation = nn.functional.tanh
        self.sentence_context = nn.Parameter(torch.randn(hidden_size,1))
    
    def forward(self, sent_outputs):
        
        o = self.linear(sent_outputs)
        o = self.activation(o)
        o = torch.matmul(o, self.sentence_context)
        o = torch.mul(o, sent_outputs)
        o = torch.sum(o, dim=1) ## Sum along the sentences
        
        return o
        

In [65]:
s_attn = SentAttn(200)

In [66]:
d_vec = s_attn(sentence_output)

/home/shikhar/.conda/envs/ml/lib/python3.6/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [71]:
d_vec.shape

torch.Size([1, 200])

In [69]:
class OutputLayer(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.input_size = input_size
        
        self.linear = nn.Linear(input_size, 1)
    
    def forward(self, input):
        return self.linear(input)
        

In [74]:
output_layer = OutputLayer(200)
output = output_layer(d_vec)

In [75]:
output

tensor([[-0.7345]], grad_fn=<AddmmBackward>)